# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
gb = GradientBoostingClassifier()
param = {                            # these were the best-performing hyperparameter values from the last model
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Justin\anaconda3\envs\harvard\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,196.990468,2.226513,0.254729,0.014819,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.968582,0.978456,0.971249,0.965858,0.967655,0.970360,0.004406,1
2,190.334366,1.677332,0.243249,0.014491,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.975763,0.970350,0.968553,0.967655,0.969642,0.003382,2
3,283.871524,3.102368,0.221330,0.010880,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.978456,0.967655,0.969452,0.966757,0.969462,0.004722,3
4,235.111264,2.057324,0.202499,0.014260,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964093,0.974865,0.968553,0.968553,0.969452,0.969104,0.003436,4
5,342.325111,1.812240,0.195062,0.024055,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964093,0.976661,0.969452,0.966757,0.968553,0.969103,0.004199,5


In [4]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators': [50, 100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Justin\anaconda3\envs\harvard\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,338.978071,2.927959,0.198860,0.020600,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.978456,0.970350,0.968553,0.969452,0.970002,0.004905,1
7,231.994069,2.994216,0.215208,0.008577,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.976661,0.967655,0.964960,0.972147,0.968924,0.004907,2
5,287.037818,1.327741,0.261699,0.017373,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.977558,0.965858,0.964960,0.969452,0.968564,0.004791,3
1,126.333858,0.798786,0.237035,0.016818,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.967655,0.963163,0.966757,0.968025,0.005471,4
4,187.509404,2.461709,0.266996,0.022518,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.975763,0.965858,0.964061,0.969452,0.967845,0.004419,5
